# Importing the Dependencies


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [6]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [7]:
# first 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [8]:
# dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11959 entries, 0 to 11958
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    11959 non-null  int64  
 1   V1      11959 non-null  float64
 2   V2      11959 non-null  float64
 3   V3      11959 non-null  float64
 4   V4      11959 non-null  float64
 5   V5      11959 non-null  float64
 6   V6      11959 non-null  float64
 7   V7      11959 non-null  float64
 8   V8      11959 non-null  float64
 9   V9      11959 non-null  float64
 10  V10     11959 non-null  float64
 11  V11     11959 non-null  float64
 12  V12     11959 non-null  float64
 13  V13     11959 non-null  float64
 14  V14     11959 non-null  float64
 15  V15     11959 non-null  float64
 16  V16     11959 non-null  float64
 17  V17     11959 non-null  float64
 18  V18     11959 non-null  float64
 19  V19     11959 non-null  float64
 20  V20     11958 non-null  float64
 21  V21     11958 non-null  float64
 22

In [9]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [10]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    11906
1.0       52
Name: Class, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [12]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [13]:
print(legit.shape)
print(fraud.shape)

(11906, 31)
(52, 31)


In [14]:
# statistical measures of the data
legit.Amount.describe()

count    11906.000000
mean        62.198127
std        177.379105
min          0.000000
25%          5.292500
50%         15.950000
75%         50.000000
max       7712.430000
Name: Amount, dtype: float64

In [15]:
fraud.Amount.describe()

count      52.000000
mean       97.724808
std       321.188775
min         0.000000
25%         1.000000
50%         1.000000
75%         1.772500
max      1809.680000
Name: Amount, dtype: float64

In [16]:
# compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,7993.389048,-0.200839,0.259753,0.929229,0.255987,-0.076685,0.149326,-0.097364,-0.053087,0.927305,-0.256049,0.794760,-1.403693,0.936354,0.827956,-0.178954,-0.003069,0.412508,-0.012325,-0.067319,0.020836,-0.063076,-0.154427,-0.032369,0.018962,0.105861,0.061825,0.005330,0.000012,62.198127
1.0,11569.615385,-3.769104,4.258027,-8.202329,6.396396,-2.342431,-1.990562,-5.740207,0.950650,-2.729769,-7.089604,6.677641,-10.357643,0.519618,-9.729181,-0.244081,-5.034546,-7.163796,-2.346145,0.250318,0.411236,0.305259,-0.021762,-0.340563,-0.406311,-0.139505,0.306967,0.692927,0.059751,97.724808


Under-Sampling


Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 52

In [31]:
legit_sample = legit.sample(n=52)

Concatenating two DataFrames

In [42]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [44]:
new_dataset.reset_index(drop=True,inplace=True)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,7029,-0.981672,1.465162,0.898816,0.229244,-0.192370,-0.580265,0.075814,0.502303,1.128964,-0.800285,0.587660,-3.095782,0.361054,1.567592,0.545859,0.232628,1.260933,-0.115020,-0.678458,-0.031567,-0.390740,-0.916208,0.103282,-0.081085,-0.231172,0.074539,0.211616,0.076662,8.99,0.0
1,2643,1.235152,0.107871,-0.068843,0.873224,0.308687,0.390010,-0.088297,0.164986,0.333870,0.022947,-1.636074,-1.012242,-1.439644,0.622509,1.541822,0.497261,-0.628104,-0.209320,-0.121652,-0.194923,-0.393477,-1.251328,0.026763,-1.429134,0.321594,-0.592952,0.023889,0.014569,24.66,0.0
2,5726,0.593305,-0.954352,-0.493094,0.861260,-0.135350,-0.018209,0.493690,-0.223737,1.245957,-0.477333,1.281992,-1.910603,1.439962,1.903475,-1.245939,0.150753,0.179190,0.523678,0.426557,0.598693,-0.004812,-0.484567,-0.535545,-0.509408,0.583817,0.513713,-0.155059,0.038810,368.00,0.0
3,19909,1.463935,-0.539306,0.274801,-0.434944,-0.740462,-0.332180,-0.757185,-0.120367,1.100284,0.162982,-0.620653,-3.820476,0.504742,1.179528,-1.021421,0.852644,1.183419,-1.061762,1.062216,-0.065586,-0.245140,-0.414189,-0.104947,-0.526130,0.603060,-0.225724,-0.021619,-0.005408,5.00,0.0
4,7019,-0.869825,0.828357,1.962723,-1.726934,0.116559,-0.456945,0.666188,-0.036267,2.095125,-2.126963,0.660880,-1.300530,2.414220,1.060239,-0.714668,-1.154527,0.897258,-0.665084,-0.179031,0.014320,-0.367981,-0.446467,-0.271398,0.044285,0.600158,-0.787099,0.311380,0.119218,7.45,0.0


In [45]:
new_dataset['Class'].value_counts()

1.0    52
0.0    52
Name: Class, dtype: int64

Splitting the data into Features & Targets


In [46]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [47]:
print(X)

      Time         V1        V2  ...       V27       V28  Amount
0     7029  -0.981672  1.465162  ...  0.211616  0.076662    8.99
1     2643   1.235152  0.107871  ...  0.023889  0.014569   24.66
2     5726   0.593305 -0.954352  ... -0.155059  0.038810  368.00
3    19909   1.463935 -0.539306  ... -0.021619 -0.005408    5.00
4     7019  -0.869825  0.828357  ...  0.311380  0.119218    7.45
..     ...        ...       ...  ...       ...       ...     ...
99   18690 -15.398845  7.472324  ...  2.262942 -1.178063    1.00
100  19762 -14.179165  7.421370  ...  2.171905 -1.395288    1.00
101  20011 -14.724627  7.875157  ...  2.161129 -1.401282    1.00
102  20332 -15.271362  8.326581  ...  2.149247 -1.406811    1.00
103  20451 -15.819179  8.775997  ...  2.136424 -1.411945    1.00

[104 rows x 30 columns]


In [48]:
print(Y)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
99     1.0
100    1.0
101    1.0
102    1.0
103    1.0
Name: Class, Length: 104, dtype: float64


Split the data into Training data & Testing Data

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=76)

In [55]:
print(X.shape, X_train.shape, X_test.shape)

(104, 30) (83, 30) (21, 30)


Model Training

Logistic Regression

In [63]:
model = svm.SVC(kernel='linear')

In [65]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Model Evaluation

In [66]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [67]:
print('Accuracy score on Training Data : ', training_data_accuracy)

Accuracy score on Training Data :  0.963855421686747


In [68]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.9523809523809523
